# Cauchy-NMF (Cauchy Non-negative Matrix Factorization)

## 因子分解

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git

In [ ]:
%cd "/content/audio_source_separation/egs/nmf-example/cauchy-nmf"

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from algorithm.nmf import CauchyNMF

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
fft_size, hop_size = 1024, 256

In [ ]:
x, sr = sf.read("../../../dataset/sample-song/sample-1_electric-guitar_8000.wav")

### 目的音源

In [ ]:
ipd.Audio(x, rate=sr)

In [ ]:
_, _, spectrogram = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)
X = np.abs(spectrogram)**2

In [ ]:
n_basis = 6

### NMFの実行

In [ ]:
np.random.seed(111)
nmf = CauchyNMF(n_basis=n_basis)

In [ ]:
basis, activation = nmf(X, iteration=20)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.show()

### 再構成信号

In [ ]:
X[X < 1e-12] = 1e-12

In [ ]:
Z = basis @ activation

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

### NMF後の各信号

In [ ]:
for idx in range(n_bbasis):
    Z = basis[:, idx: idx+1] * activation[idx: idx+1, :]

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_basis):
    estimated_spectrogram = basis[:, idx: idx + 1] @ activation[idx: idx + 1, :]

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()

## 更新アルゴリズム
- `'naive-multipricative'`: Naive multipricative update
- `'mm'`: Majorization minimization アルゴリズム
- `'me'`: Majorization equalization アルゴリズム
- `'mm_fast'`: 高速化 majorization minimization アルゴリズム

### Naive multipricative

In [ ]:
np.random.seed(111)
nmf_naive = CauchyNMF(n_basis=n_basis, algorithm='naive-multipricative')
basis, activation = nmf_naive(X, iteration=20)

### MMアルゴリズム

In [ ]:
np.random.seed(111)
nmf_mm = CauchyNMF(n_basis=n_basis, algorithm='mm')
basis, activation = nmf_mm(X, iteration=20)

### MEアルゴリズム

In [ ]:
np.random.seed(111)
nmf_me = CauchyNMF(n_basis=n_basis, algorithm='me')
basis, activation = nmf_me(X, iteration=20)

### 高速化MMアルゴリズム

In [ ]:
np.random.seed(111)
nmf_mm_fast = CauchyNMF(n_basis=n_basis, algorithm='mm_fast')
basis, activation = nmf_mm_fast(X, iteration=20)

In [ ]:
plt.figure()
plt.plot(nmf_naive.loss, color='mediumvioletred', label='Naive multipricative')
plt.plot(nmf_mm.loss, color='mediumblue', label='MM algorithm')
plt.plot(nmf_me.loss, color='mediumseagreen', label='ME algorithm')
plt.plot(nmf_mm_fast.loss, linestyle='dashed', color='mediumseagreen', label='Fast MM algorithm')

plt.legend()
plt.show()